<a href="https://colab.research.google.com/github/lorenafc/Capita_Selecta_RHI50403/blob/main/Maps_authors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import libraries
import pandas as pd
import geopandas as gpd
import folium
from folium import Marker
import matplotlib.pyplot as plt


In [2]:
!pip install cartopy -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 17.9 MB/s eta 0:00:00


In [3]:
import cartopy.crs as ccrs

In [4]:
!pip install contextily -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 19.9 MB/s eta 0:00:00


In [5]:
!pip install APNG -q

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from google.colab import files
uploaded=files.upload()

Saving merged_additional_authors_encoded.xlsx to merged_additional_authors_encoded.xlsx


In [8]:
file_name = 'merged_additional_authors_encoded.xlsx'
authors_small_cities = pd.read_excel(file_name, engine='openpyxl')
authors_small_cities_copy=authors_small_cities.copy()
authors_small_cities_copy.head()

,Unnamed: 0,starturl,bitrhyear,deathyear,nameandbirthdeathyear,georeferenceurl,borncity,deathcity,activecity,latitude_born,longitude_born,latitude_death,longitude_death,latitude_active,longitude_active
0,0,http://viaf.org/viaf/81048146,1570,1634.0,"Carneiro, Tomé Tavares 1570-1634",viafprocessedfile,,NaN,Rio Tinto,NaN,NaN,NaN,NaN,-6.803828,-35.077569
1,1,http://viaf.org/viaf/180581759,NaN,1775.0,"Kolíček, Caesarij, -1775",viafprocessedfile,,Cieszin,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,http://viaf.org/viaf/100161519,NaN,1570.0,"Hertel, Jakob 15..?-1570",http://catalogue.bnf.fr/ark:/12148/cb15011774s,(Chur,NaN,NaN,46.854747,9.526490,NaN,NaN,NaN,NaN
3,3,http://viaf.org/viaf/169350346,1699,1770.0,"Ferro Caaveyro, Lucas Antonio ca. 1699-1770",www.wikidata.org/wiki/Q3393623#sitelinks-wikip...,A Capela,NaN,NaN,43.442068,-8.043400,NaN,NaN,NaN,NaN
4,4,http://viaf.org/viaf/67541728,1502,1600.0,"Aparicio, Sebastián de 1502-1600 Santo",www.wikidata.org/wiki/Q2486114#sitelinks-wikip...,A Gudiña,Puebla City,NaN,42.060548,-7.140352,19.04372,-98.198149,NaN,NaN


In [9]:
# Convert the active city locations to a GeoDataFrame
authors_small_cities.rename(columns={'Unnamed: 0': 'indexauthor'}, inplace=True)
authors_small_cities_active = authors_small_cities.drop(['borncity', 'deathcity', "latitude_born", "longitude_born", "latitude_death", "longitude_death"], axis=1)

authors_small_cities_active = gpd.GeoDataFrame(
    authors_small_cities_active,
    geometry=gpd.points_from_xy(authors_small_cities.longitude_active, authors_small_cities.latitude_active)
)
authors_small_cities_active.crs = "EPSG:4326" # WGS84 Latitude/Longitude

In [10]:
# Filter out entries with 'POINT EMPTY' geometries
authors_small_cities_active = authors_small_cities_active[authors_small_cities_active['geometry'].is_valid & ~authors_small_cities_active['geometry'].is_empty]


In [11]:
authors_small_cities_active.head()

,indexauthor,starturl,bitrhyear,deathyear,nameandbirthdeathyear,georeferenceurl,activecity,latitude_active,longitude_active,geometry
0,0,http://viaf.org/viaf/81048146,1570,1634.0,"Carneiro, Tomé Tavares 1570-1634",viafprocessedfile,Rio Tinto,-6.803828,-35.077569,POINT (-35.07757 -6.80383)
10,10,http://viaf.org/viaf/24946396,1706,1761.0,"Bahil, Matthias, 1706-1761",http://d-nb.info/gnd/105529015X,Prešov,49.000007,21.239212,POINT (21.23921 49.00001)
17,17,http://viaf.org/viaf/83653553,1716,1776.0,Jan Ondřej Kayser z Kaysernu,http://d-nb.info/gnd/1075812674,Königgrätz,50.209211,15.832751,POINT (15.83275 50.20921)
19,19,http://viaf.org/viaf/54553192,1618,1678.0,"Lagus, Daniel, 1618-1678",http://d-nb.info/gnd/103085491,Kolberg,54.176068,15.576554,POINT (15.57655 54.17607)
30,30,http://viaf.org/viaf/220785973,NaN,1743.0,"Schräber, Andreas -1743",http://d-nb.info/gnd/1017598924,Żary,51.624438,15.148725,POINT (15.14872 51.62444)


#TESTING WITH A RANGE OF 100 YEARS - 800 to 900

In [12]:
print(authors_small_cities_active.shape)

# Filter the DataFrame for the specified year range
authors_small_cities_active = authors_small_cities_active[
    (authors_small_cities_active['deathyear'] >= 800) & (authors_small_cities_active['deathyear'] <= 900)
].copy()

print(authors_small_cities_active.shape)

# Display the first few rows of the new filtered DataFrame
print(authors_small_cities_active.head(15))

(46236, 10)
(210, 10)
       indexauthor                                  starturl bitrhyear  \
4813          4813            http://viaf.org/viaf/316446895       788   
22247        22247            http://viaf.org/viaf/300991153       NaN   
24153        24153             http://viaf.org/viaf/76708720       830   
24562        24562            http://viaf.org/viaf/316420894       NaN   
24844        24844             http://viaf.org/viaf/89101819       NaN   
24845        24845             http://viaf.org/viaf/80796533       NaN   
25496        25496             http://viaf.org/viaf/22949431       840   
25579        25579             http://viaf.org/viaf/84482363       NaN   
25580        25580              http://viaf.org/viaf/7777732       NaN   
25834        25834              http://viaf.org/viaf/3268904       730   
25890        25890            http://viaf.org/viaf/305193436       NaN   
26096        26096            http://viaf.org/viaf/316406114       NaN   
26097        260

In [ ]:
print(cx.providers.keys())
print(cx.providers.values())



In [45]:
#source: Lucas Koren
#this code shows only the points,no map

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as cx
import os
from apng import APNG

# Define the start and end years
startyear = 800
endyear = 820
step = 1

output_dir = "/content/drive/My Drive/capita_selecta/active_maps/"
os.makedirs(output_dir, exist_ok=True)

# Loop through the timesteps
for i, timestep in enumerate(range(startyear - 1, endyear, step)):
    # Create a string to represent the current timestep
    years = f"{timestep + 1}"

    # Filter the GeoDataFrame for authors active in the timestep
    authors_active_timestep = authors_small_cities_active[
        (authors_small_cities_active['deathyear'] > timestep) &
        (authors_small_cities_active['deathyear'] <= timestep + step)
    ]

    # Check if there are any valid records for the current timestep
    if not authors_active_timestep.empty:
        # Group by unique points and get the count for each group and turn into geodataframe
        unique_points_gdf = gpd.GeoDataFrame(
            authors_active_timestep.groupby('geometry').size().reset_index(name='point_count'),
            geometry='geometry'
        )

        # Ensure the CRS is set to EPSG:4326 for latitude and longitude
        unique_points_gdf = unique_points_gdf.set_crs(epsg=4326, allow_override=True)

        # Set up plot
        fig, ax = plt.subplots(figsize=(19, 10))
        ax.set_xlim(-180, 180)
        ax.set_ylim(-65, 70)

        # Set title and axis labels
        ax.set_title(f"Authors' Hotspots ({years})")

        # Set labels for x and y axis
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')

        # Add basemap
        cx.add_basemap(ax, attribution=False, zoom=1, crs=unique_points_gdf.crs.to_string(), source=cx.providers.OpenStreetMap.Mapnik)#CartoDB.PositronNoLabels)

        #ax.set_global()

        # Plot the points
        unique_points_gdf.plot(ax=ax, column='point_count', cmap='viridis', markersize=unique_points_gdf['point_count']*15) # Adjust markersize as needed

        # Save plot as .png file
        plt.savefig(f"{output_dir}active_hotspot_plot_{years}.png")

        # Close the current figure to avoid overlapping plots
        plt.close(fig)



# Create an animated gif from the saved images

output_dir = "/content/drive/My Drive/capita_selecta/active_maps/"
os.makedirs(output_dir, exist_ok=True)

active_files = os.listdir(output_dir)
active_files_paths = [os.path.join(output_dir, active_file) for active_file in active_files if active_file.endswith('.png')]

# source https://stackoverflow.com/questions/753190/programmatically-generate-video-or-animated-gif-in-python

import imageio
images = []
for active_file in active_files_paths: # Use active_files_paths instead of active_files
    images.append(imageio.imread(active_file))
imageio.mimsave('/content/drive/My Drive/capita_selecta/active_maps/authors_movie_m15_zoom1_set_global.gif', images)



/usr/local/lib/python3.10/dist-packages/geopandas/tools/hilbert_curve.py:106: RuntimeWarning: divide by zero encountered in scalar divide
  res = (vals - val_range[0]) * (n / width)
/usr/local/lib/python3.10/dist-packages/geopandas/tools/hilbert_curve.py:106: RuntimeWarning: invalid value encountered in multiply
  res = (vals - val_range[0]) * (n / width)
/usr/local/lib/python3.10/dist-packages/geopandas/tools/hilbert_curve.py:109: RuntimeWarning: invalid value encountered in cast
  return res.astype(np.uint32)
/usr/local/lib/python3.10/dist-packages/geopandas/tools/hilbert_curve.py:106: RuntimeWarning: divide by zero encountered in scalar divide
  res = (vals - val_range[0]) * (n / width)
/usr/local/lib/python3.10/dist-packages/geopandas/tools/hilbert_curve.py:106: RuntimeWarning: invalid value encountered in multiply
  res = (vals - val_range[0]) * (n / width)
/usr/local/lib/python3.10/dist-packages/geopandas/tools/hilbert_curve.py:109: RuntimeWarning: invalid value encountered in ca